In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import numpy as np

path = "../../../dane/8CPU_20RAM/3600s/4_merged.csv"

In [2]:
# Załaduj dane
df = pd.read_csv(path)

In [3]:
for col in ['replicaId', 'endpointUrl_methods']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [4]:
# Dodajemy skalowanie dla wybranych kolumn
features_to_scale = ['queueSizeForward_methods', 'queueSizeBack_methods',
                     'cpuUsage_stock', 'memoryUsage_stock',
                     'applicationTime_trading', 'databaseTime_trading',
                     'numberOfSellOffers_trading', 'numberOfBuyOffers_trading']
scaler = StandardScaler()
for col in features_to_scale:
    df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

In [5]:
# Wybór cech
features = [
            # 'timestamp',
            # 'apiTime_methods',
            # 'applicationTime_methods',
            # 'databaseTime_methods',
            'endpointUrl_methods',
            'queueSizeForward_methods',
            'queueSizeBack_methods',
            'cpuUsage_stock',
            'memoryUsage_stock',
            'applicationTime_trading',
            'databaseTime_trading',
            'numberOfSellOffers_trading',
            'numberOfBuyOffers_trading',
            # 'cpuUsage_traffic',
            # 'memoryUsage_traffic',
            # 'replicaId'
            ]
df_features = df[features]

# Kodowanie pudełek
le = LabelEncoder()
df['test'] = le.fit_transform(df['test'])

# Kopiowanie danych
df_encoded = df_features.copy()
df_encoded['test'] = df['test']

In [6]:
# Stwórz puste listy do przechowywania danych treningowych i testowych
X_train = []
X_test = []
y_train = []
y_test = []

In [7]:
def create_windows(X, y, window_size, step_size):
    X_windows = []
    y_windows = []

    # Przesuń okno po danych
    for i in range(0, len(X) - window_size, step_size):
        # Utwórz okno danych
        X_window = X.iloc[i:i + window_size]
        # Utwórz etykietę dla okna (etykieta ostatniej obserwacji w oknie)
        y_window = y.iloc[i + window_size]

        X_windows.append(X_window.values)
        y_windows.append(y_window)

    return np.array(X_windows), np.array(y_windows)

In [8]:
window_size = 700
step_size = 100
# Dla każdego unikalnego pudełka
for box in df_encoded['test'].unique():
    # Wybierz tylko rekordy dla tego pudełka
    box_data = df_encoded[df_encoded['test'] == box]
    
    # Oblicz punkt podziału (80% danych)
    split_point = int(len(box_data) * 0.8)
    
    # Dodaj pierwsze 80% rekordów do danych treningowych
    X_train_box = box_data.drop('test', axis=1).iloc[:split_point]
    y_train_box = box_data['test'].iloc[:split_point]
    
    # Dodaj ostatnie 20% rekordów do danych testowych
    X_test_box = box_data.drop('test', axis=1).iloc[split_point:]
    y_test_box = box_data['test'].iloc[split_point:]
    
    # Stwórz okienka dla danych treningowych
    X_train_windows, y_train_windows = create_windows(X_train_box, y_train_box, window_size, step_size)
    
    # Stwórz okienka dla danych testowych
    X_test_windows, y_test_windows = create_windows(X_test_box, y_test_box, window_size, step_size)
    
    # Dodaj okienka do list
    X_train.append(X_train_windows)
    y_train.append(y_train_windows)
    X_test.append(X_test_windows)
    y_test.append(y_test_windows)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from keras.utils import to_categorical

In [10]:
# Połączenie danych treningowych dla wszystkich pudełek
X_train_combined = np.concatenate(X_train, axis=0)
y_train_combined = np.concatenate(y_train, axis=0)

In [11]:
# Zakodowanie etykiet w formacie one-hot encoding
y_train_combined_encoded = to_categorical(y_train_combined, num_classes=len(df['test'].unique()))

In [16]:
# dodanie szumu
noise_factor = 0.05
X_train_noisy = X_train_combined + noise_factor * np.random.normal(size=X_train_combined.shape)

In [17]:
# Trenowanie modelu na połączonym zestawie danych
model = Sequential()    
model.add(LSTM(100, input_shape=(window_size, X_train_combined.shape[2])))
model.add(Dense(len(df['test'].unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_combined, y_train_combined_encoded, epochs=5, batch_size=32)

Epoch 1/5
552/552 [==============================] - 194s 347ms/step - loss: 0.5231 - accuracy: 0.7862
Epoch 2/5
552/552 [==============================] - 199s 360ms/step - loss: 0.2153 - accuracy: 0.9394
Epoch 3/5
552/552 [==============================] - 194s 351ms/step - loss: 0.1183 - accuracy: 0.9655
Epoch 4/5
552/552 [==============================] - 158s 285ms/step - loss: 0.1275 - accuracy: 0.9588
Epoch 5/5
552/552 [==============================] - 166s 300ms/step - loss: 0.4222 - accuracy: 0.8262


In [18]:
for i in range(len(X_test)):
    y_test_encoded = to_categorical(y_test[i], num_classes=len(df['test'].unique()))
    loss, accuracy = model.evaluate(X_test[i], y_test_encoded)
    print(f"Test {i+1}: Loss = {loss}, Accuracy = {accuracy}")

31/31 [==============================] - 4s 105ms/step - loss: 0.2093 - accuracy: 0.8739
Test 1: Loss = 0.20926272869110107, Accuracy = 0.8738555312156677
29/29 [==============================] - 3s 108ms/step - loss: 0.0040 - accuracy: 1.0000
Test 2: Loss = 0.003981402609497309, Accuracy = 1.0
39/39 [==============================] - 4s 103ms/step - loss: 0.3453 - accuracy: 0.9739
Test 3: Loss = 0.34528258442878723, Accuracy = 0.9739201068878174
40/40 [==============================] - 4s 105ms/step - loss: 0.0023 - accuracy: 1.0000
Test 4: Loss = 0.0023167231120169163, Accuracy = 1.0
